Prueba tus estrategias de trading con Backtrader
García-Ferreira
https://www.garcia-ferreira.es/backtesting-con-backtrader/

In [2]:
#!pip3 install backtrader
#!pip3 install yahoo_fin

In [1]:
import backtrader as bt

In [7]:
from yahoo_fin.stock_info import get_data
starbucks= get_data("sbux", 
   start_date="01/01/2015", 
   end_date="01/01/2022", 
   index_as_date=True, 
   interval="1d")

In [8]:
starbucks

,open,high,low,close,adjclose,volume,ticker
2015-01-02,41.064999,41.490002,40.445000,40.720001,34.589939,6886000,SBUX
2015-01-05,40.070000,40.334999,39.744999,39.939999,33.927357,11623800,SBUX
2015-01-06,40.169998,40.195000,39.279999,39.615002,33.651287,7664400,SBUX
2015-01-07,39.875000,40.615002,39.700001,40.590000,34.479507,9732600,SBUX
2015-01-08,41.165001,41.650002,41.009998,41.244999,35.035904,13170600,SBUX
...,...,...,...,...,...,...,...
2021-12-27,112.599998,114.370003,112.389999,114.220001,109.876205,3716700,SBUX
2021-12-28,114.000000,116.279999,113.809998,115.570000,111.174873,4561600,SBUX
2021-12-29,115.400002,116.959999,115.309998,116.379997,111.954079,4215300,SBUX
2021-12-30,116.379997,117.339996,116.080002,116.239998,111.819405,4068700,SBUX


In [9]:
class cruce50_200(bt.Strategy):
    def __init__(self):
        # Inicializamos la media de 50 y la de 200
        self.sma50 = bt.indicators.SimpleMovingAverage(self.data, period=50, plotname="50 SMA")
        self.sma200 = bt.indicators.SimpleMovingAverage(self.data, period=200, plotname="200 SMA")
        self.cruce = bt.ind.CrossOver(self.sma50,self.sma200)
        # Inicializamos la variable order, que usaremos para el control de las ordenes
        # Para mantener el seguimiento de las órdenes pendientes
        self.order = None
        self.buyprice = None
        self.sellprice = None
        # guardamos el último dato de cierre
        self.dataclose = self.datas[0].close
    def next(self):
        # Comprobamos si estamos en el mercado
        # si no lo estamos seguimos para adelante
        if not self.position:
            # Si la media de 50 es mayor que la media de 200 compramos
            # print("{}: {} - {}".format(self.datas[0].datetime.date(0), self.sma50[0], self.sma200[0]))
            if self.cruce > 0:
                self.order = self.buy()
                self.log('Orden de compra lanzada: %.2f' % self.dataclose[0])
        # Si estamos en el mercado
        else:
            if self.cruce < 0:
                # Si es así vendemos (con todos los parametros por defecto posibles)
                self.log('Orden de venta lanzada: %.2f' % self.dataclose[0])
          
                # Mantenemos un seguimiento de la orden para evitar abrir una segunda orden
                self.order = self.sell()
    # Creamos este método para el control de las ordenes
    def notify_order(self, order):
        # Orden de compra o de venta aceptada por el brocker - Nada que hacer
        if order.status in [order.Submitted, order.Accepted]:
            return
        # Orden de compra completada
        # Comprobamos si es de compra o de venta y mostramos los resultados
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                self.log('COMPRA EJECUTADA [Precio: %.2f, Comisión: %.2f]' % (order.executed.price, order.executed.comm))
            elif order.issell():
                self.sellprice = order.executed.price
                self.log('VENTA EJECUTADA [Precio: %.2f, Comisión: %.2f]' % (order.executed.price, order.executed.comm))
        # Orden de compra cancelada
        elif order.status == order.Canceled:
            self.log('Orden Cancelada')
        # Orden de compra cancelada por el margen de tu dinero
        elif order.status == order.Margin:
            self.log('Orden de Margen')
        # Orden de compra rechazada
        elif order.status == order.Rejected:
            self.log('Orden Rechazada')
    # Metodo para mostrar de manera más fácil los logs por pantalla
    # incluyendo la fecha en la que se producen los eventos
    def log(self, txt, dt=None):
        # Vamos mostrando los datos
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

In [10]:
cerebro = bt.Cerebro(stdstats=False)

# Añadimos los datos que hemso descargado previamente
data1 = bt.feeds.PandasData(dataname=starbucks)
cerebro.adddata(data1)

# Añadimos la cantidad inicial de dinero con la que vamos a realizar el trading
cerebro.broker.setcash(10000.0)
# Añadimos la comisión - 0.1%
cerebro.broker.setcommission(commission=0.001)
# Tamaño de los lotes que queremos comprar
cerebro.addsizer(bt.sizers.FixedSize, stake=1)
cerebro.addstrategy(cruce50_200)
# Mostramos los valores tanto inicial como final durante la ejecución del
# proceso de backtesting
print('Valor inicial del portfolio: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Valor final del porfolio: %.2f' % cerebro.broker.getvalue())

Valor inicial del portfolio: 10000.00
2017-01-06, Orden de compra lanzada: 57.13
2017-01-09, COMPRA EJECUTADA [Precio: 57.26, Comisión: 0.06]
2017-08-17, Orden de venta lanzada: 53.04
2017-08-18, VENTA EJECUTADA [Precio: 52.92, Comisión: 0.05]
2018-01-09, Orden de compra lanzada: 59.18
2018-01-10, COMPRA EJECUTADA [Precio: 60.00, Comisión: 0.06]
2018-06-25, Orden de venta lanzada: 50.66
2018-06-26, VENTA EJECUTADA [Precio: 50.51, Comisión: 0.05]
2018-10-30, Orden de compra lanzada: 58.59
2018-10-31, COMPRA EJECUTADA [Precio: 58.98, Comisión: 0.06]
2020-03-05, Orden de venta lanzada: 76.19
2020-03-06, VENTA EJECUTADA [Precio: 73.46, Comisión: 0.07]
2020-09-16, Orden de compra lanzada: 88.38
2020-09-17, COMPRA EJECUTADA [Precio: 87.05, Comisión: 0.09]
2021-11-18, Orden de venta lanzada: 112.90
2021-11-19, VENTA EJECUTADA [Precio: 112.73, Comisión: 0.11]
Valor final del porfolio: 10025.78


In [11]:
cerebro.plot(style='candle', 
             iplot=False, 
             volume = True, 
             barupfill = False, 
             bardownfill = False, 
             barup='green', 
             bardown='red')

[[<Figure size 1416x596 with 3 Axes>]]